# Laying the foundation for an Open Data catalog

Although I could do the exercise in .py file, I decided to do it in a Jupyter Notebook because I can describe it better.

The first step is download the statistical data from the ine website.

In [1]:
import requests
import time

start_time = time.time()

url = 'https://www.ine.es/censos2011_datos/indicadores_seccion_censal_csv.zip'

r = requests.get(url)

with open('data/censo.zip', 'wb') as f:
    f.write(r.content)
    
print("{} seconds".format(time.time() - start_time))

3.5538923740386963 seconds


Like the file is compressed, we need to unzipped. I use the zipfile lib to do it.

In [2]:
import zipfile
with zipfile.ZipFile('data/censo.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

I repeat it with the geometry data

In [3]:
start_time = time.time()

url = 'https://www.ine.es/censos2011_datos/cartografia_censo2011_nacional.zip'

r = requests.get(url)

with open('data/shapefiles.zip', 'wb') as f:
    f.write(r.content)
    
print("{} seconds".format(time.time() - start_time))

42.34533905982971 seconds


In [4]:
import zipfile
with zipfile.ZipFile('data/shapefiles.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

It's the first time that I use postgres. To use it with python, I decided use *psycopg2* library. First, I create the table that I have called *indicators*. I have created all the columns that appear in the description file.

In [6]:
import psycopg2

commands = """
        CREATE TABLE indicators
        (
            id SERIAL,
            ccaa INTEGER NOT NULL,
            cpro INTEGER NOT NULL,
            cmum INTEGER NOT NULL,
            dist INTEGER NOT NULL,
            secc INTEGER NOT NULL,
            t1_1 INTEGER,
            t2_1 INTEGER,
            t2_2 INTEGER,
            t3_1 INTEGER,
            t3_2 INTEGER,
            t3_3 INTEGER,
            t4_1 INTEGER,
            t4_2 INTEGER,
            t4_3 INTEGER,
            t4_4 INTEGER,
            t4_5 INTEGER,
            t4_6 INTEGER,
            t4_7 INTEGER,
            t4_8 INTEGER,
            t5_1 INTEGER,
            t5_2 INTEGER,
            t5_3 INTEGER,
            t5_4 INTEGER,
            t5_5 INTEGER,
            t5_6 INTEGER,
            t5_7 INTEGER,
            t5_8 INTEGER,
            t5_9 INTEGER,
            t5_10 INTEGER,
            t5_11 INTEGER,
            t5_12 INTEGER,
            t5_13 INTEGER,
            t5_14 INTEGER,
            t5_15 INTEGER,
            t5_16 INTEGER,
            t6_1 INTEGER,
            t6_2 INTEGER,
            t7_1 INTEGER,
            t7_2 INTEGER,
            t7_3 INTEGER,
            t7_4 INTEGER,
            t7_5 INTEGER,
            t7_6 INTEGER,
            t8_1 INTEGER,
            t8_2 INTEGER,
            t8_3 INTEGER,
            t8_4 INTEGER,
            t9_1 INTEGER,
            t9_2 INTEGER,
            t9_3 INTEGER,
            t9_4 INTEGER,
            t9_5 INTEGER,
            t9_6 INTEGER,
            t10_1 INTEGER,
            t10_2 INTEGER,
            t10_3 INTEGER,
            t10_4 INTEGER,
            t10_5 INTEGER,
            t11_1 INTEGER,
            t11_2 INTEGER,
            t11_3 INTEGER,
            t11_4 INTEGER,
            t11_5 INTEGER,
            t11_6 INTEGER,
            t11_7 INTEGER,
            t11_8 INTEGER,
            t11_9 INTEGER,
            t11_10 INTEGER,
            t11_11 INTEGER,
            t11_12 INTEGER,
            t11_13 INTEGER,
            t11_14 INTEGER,
            t11_15 INTEGER,
            t12_1 INTEGER,
            t12_2 INTEGER,
            t12_3 INTEGER,
            t12_4 INTEGER,
            t12_5 INTEGER,
            t12_6 INTEGER,
            t13_1 INTEGER,
            t13_2 INTEGER,
            t13_3 INTEGER,
            t13_4 INTEGER,
            t13_5 INTEGER,
            t13_6 INTEGER,
            t13_7 INTEGER,
            t13_8 INTEGER,
            t13_9 INTEGER,
            t13_10 INTEGER,
            t13_11 INTEGER,
            t13_12 INTEGER,
            t14_1 INTEGER,
            t14_2 INTEGER,
            t14_3 INTEGER,
            t14_4 INTEGER,
            t14_5 INTEGER,
            t14_6 INTEGER,
            t14_7 INTEGER,
            t14_8 INTEGER,
            t14_9 INTEGER,
            t14_10 INTEGER,
            t14_11 INTEGER,
            t14_12 INTEGER,
            t15_2 INTEGER,
            t15_3 INTEGER,
            t15_5 INTEGER,
            t15_6 INTEGER,
            t15_8 INTEGER,
            t15_9 INTEGER,
            t15_11 INTEGER,
            t15_12 INTEGER,
            t15_14 INTEGER,
            t15_15 INTEGER,
            t15_16 INTEGER,
            t16_1 INTEGER,
            t17_1 INTEGER,
            t17_2 INTEGER,
            t17_3 INTEGER,
            t18_1 INTEGER,
            t18_2 INTEGER,
            t18_3 INTEGER,
            t18_4 INTEGER,
            t18_5 INTEGER,
            t18_6 INTEGER,
            t19_1 INTEGER,
            t19_2 INTEGER,
            t19_3 INTEGER,
            t19_4 INTEGER,
            t19_5 INTEGER,
            t19_6 INTEGER,
            t19_7 INTEGER,
            t19_8 INTEGER,
            t19_9 INTEGER,
            t19_10 INTEGER,
            t20_1 INTEGER,
            t20_2 INTEGER,
            t20_3 INTEGER,
            t20_4 INTEGER,
            t20_5 INTEGER,
            t20_6 INTEGER,
            t20_7 INTEGER,
            t20_8 INTEGER,
            t20_9 INTEGER,
            t21_1 INTEGER,
            t22_1 INTEGER,
            t22_2 INTEGER,
            t22_3 INTEGER,
            t22_4 INTEGER,
            t22_5 INTEGER,
            t22_6 INTEGER,
            CONSTRAINT indicators_pkey PRIMARY KEY (id)
        ) """
    
conn = None
try:
    # We connect to the PostgreSQL server
    conn = psycopg2.connect(host="localhost",database="carto", user="postgres", password="postgres")
    cur = conn.cursor()
    # We create the table
    cur.execute(commands)
    # We close the connection
    cur.close()
    # Finally, we commit the changes
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()

We insert the statistical data using the COPY postgress command. To save code, I use a for loop.

In [8]:
for ccaa in range(1,20):
    if ccaa <10:
        command = """
COPY indicators(ccaa, cpro, cmum, dist, secc, t1_1, t2_1, t2_2, t3_1, t3_2, t3_3, t4_1, t4_2, t4_3, t4_4, t4_5, t4_6, t4_7, \
             t4_8, t5_1, t5_2, t5_3, t5_4, t5_5, t5_6, t5_7, t5_8, t5_9, t5_10, t5_11, t5_12, t5_13, t5_14, t5_15, \
             t5_16, t6_1, t6_2, t7_1, t7_2, t7_3, t7_4, t7_5, t7_6, t8_1, t8_2, t8_3, t8_4, t9_1, t9_2, t9_3, t9_4, t9_5,\
             t9_6, t10_1, t10_2, t10_3, t10_4, t10_5, t11_1, t11_2, t11_3, t11_4, t11_5, t11_6, t11_7, t11_8, t11_9, \
             t11_10, t11_11, t11_12, t11_13, t11_14, t11_15, t12_1, t12_2, t12_3, t12_4, t12_5, t12_6, t13_1, t13_2, \
             t13_3, t13_4, t13_5, t13_6, t13_7, t13_8, t13_9, t13_10, t13_11, t13_12, t14_1, t14_2, t14_3, t14_4, t14_5, \
             t14_6, t14_7, t14_8, t14_9, t14_10, t14_11, t14_12, t15_2, t15_3, t15_5, t15_6, t15_8, t15_9, t15_11, \
             t15_12, t15_14, t15_15, t15_16, t16_1, t17_1, t17_2, t17_3, t18_1, t18_2, t18_3, t18_4, t18_5, t18_6, \
             t19_1, t19_2, t19_3, t19_4, t19_5, t19_6, t19_7, t19_8, t19_9, t19_10, t20_1, t20_2, t20_3, t20_4, t20_5, \
             t20_6, t20_7, t20_8, t20_9, t21_1, t22_1, t22_2, t22_3, t22_4, t22_5, t22_6) 
FROM '/home/alfonso/proyectos/tests/test-coding-skills/data/C2011_ccaa0""" + str(ccaa) + """_Indicadores.csv' DELIMITER ',' CSV HEADER; """
    else:
        command = """
COPY indicators(ccaa, cpro, cmum, dist, secc, t1_1, t2_1, t2_2, t3_1, t3_2, t3_3, t4_1, t4_2, t4_3, t4_4, t4_5, t4_6, t4_7, \
             t4_8, t5_1, t5_2, t5_3, t5_4, t5_5, t5_6, t5_7, t5_8, t5_9, t5_10, t5_11, t5_12, t5_13, t5_14, t5_15, \
             t5_16, t6_1, t6_2, t7_1, t7_2, t7_3, t7_4, t7_5, t7_6, t8_1, t8_2, t8_3, t8_4, t9_1, t9_2, t9_3, t9_4, t9_5,\
             t9_6, t10_1, t10_2, t10_3, t10_4, t10_5, t11_1, t11_2, t11_3, t11_4, t11_5, t11_6, t11_7, t11_8, t11_9, \
             t11_10, t11_11, t11_12, t11_13, t11_14, t11_15, t12_1, t12_2, t12_3, t12_4, t12_5, t12_6, t13_1, t13_2, \
             t13_3, t13_4, t13_5, t13_6, t13_7, t13_8, t13_9, t13_10, t13_11, t13_12, t14_1, t14_2, t14_3, t14_4, t14_5, \
             t14_6, t14_7, t14_8, t14_9, t14_10, t14_11, t14_12, t15_2, t15_3, t15_5, t15_6, t15_8, t15_9, t15_11, \
             t15_12, t15_14, t15_15, t15_16, t16_1, t17_1, t17_2, t17_3, t18_1, t18_2, t18_3, t18_4, t18_5, t18_6, \
             t19_1, t19_2, t19_3, t19_4, t19_5, t19_6, t19_7, t19_8, t19_9, t19_10, t20_1, t20_2, t20_3, t20_4, t20_5, \
             t20_6, t20_7, t20_8, t20_9, t21_1, t22_1, t22_2, t22_3, t22_4, t22_5, t22_6) 
FROM '/home/alfonso/proyectos/tests/test-coding-skills/data/C2011_ccaa""" + str(ccaa) + """_Indicadores.csv' DELIMITER ',' CSV HEADER; """

    conn = None
    try:
        conn = psycopg2.connect(host="localhost",database="carto", user="postgres", password="postgres")
        cur = conn.cursor()
        cur.execute(command)
        cur.close()
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

To treat the geometric data, I use the suggested ogr2ogr command. To run it, although it's not the cleanest way, the os library.

In [9]:
import os
command = "ogr2ogr -f PostgreSQL PG:\"dbname='carto' host='localhost' port='5432' user='postgres' password='postgres'\" \"/home/alfonso/proyectos/tests/test-coding-skills/data/SECC_CPV_E_20111101_01_R_INE.shp\" -nlt MultiPolygon -lco PRECISION=no"
os.system(command)

0

To give a simpler name to the table, I decided to change it to *geometry*

In [10]:
command = """
ALTER TABLE secc_cpv_e_20111101_01_r_ine
RENAME TO geometry;
"""

conn = None
try:
    conn = psycopg2.connect(host="localhost",database="carto", user="postgres", password="postgres")
    cur = conn.cursor()
    cur.execute(command)
    cur.close()
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()

Finally, we do it the SQL queries. The exercise ask for combine the data and I decided to do it whith a JOIN command.

* **Get the population density of each of the municipalities of Madrid**. I decided to limit the exit to get a shorter answer. To ger all the municipalities, we have to eliminate the "LIMIT 10" command. The shape_are is in meters, so we have to multiply by 1000000 to convert to population/km&#x00B2;.

* **Get the names of the 10 provinces with the highest percentage of people with university degrees (third-level studies)**. The indicators are INT type, so the divide operation truncate the solucion like integral number (0). For this reason, we convert the indicators to decimal.

In [17]:
command = """
SELECT
   geometry.NMUN AS name,
   SUM(indicators.t1_1)/SUM(geometry.Shape_area)*1000000 AS density
FROM
   indicators
JOIN geometry ON indicators.ccaa = CAST (geometry.CCA AS INTEGER) 
    AND indicators.cpro = CAST (geometry.CPRO AS INTEGER)
    AND indicators.cmum = CAST (geometry.CMUN AS INTEGER)
    AND indicators.dist = CAST (geometry.CDIS AS INTEGER)
    AND indicators.secc = CAST (geometry.CSEC AS INTEGER)
WHERE geometry.NPRO = 'Madrid'
GROUP BY geometry.NMUN
ORDER BY density DESC
LIMIT 10;
"""

conn = None
try:
    conn = psycopg2.connect(host="localhost",database="carto", user="postgres", password="postgres")
    cur = conn.cursor()
    cur.execute(command)
    res = cur.fetchall() 
    
    for row in res:
        print("Municipality = {}".format(row[0]) )
        print("Density = {:0.2f} persons\km2\n".format(row[1]) )
    
    cur.close()
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()

Municipality = Coslada
Density = 7382.31 persons\km2

Municipality = Madrid
Density = 5271.80 persons\km2

Municipality = Móstoles
Density = 5239.23 persons\km2

Municipality = Fuenlabrada
Density = 5025.09 persons\km2

Municipality = Alcorcón
Density = 4961.71 persons\km2

Municipality = Parla
Density = 4877.59 persons\km2

Municipality = Leganés
Density = 4280.38 persons\km2

Municipality = Torrejón de Ardoz
Density = 3790.63 persons\km2

Municipality = Alcobendas
Density = 2430.77 persons\km2

Municipality = Collado Villalba
Density = 2409.94 persons\km2



In [18]:
command = """
SELECT
   geometry.NPRO AS name,
   SUM(CAST(indicators.t12_5 AS DECIMAL))/SUM(CAST(indicators.t1_1 AS DECIMAL))*100 AS percentage
FROM
   indicators
JOIN geometry ON indicators.ccaa = CAST (geometry.CCA AS INTEGER) 
    AND indicators.cpro = CAST (geometry.CPRO AS INTEGER)
    AND indicators.cmum = CAST (geometry.CMUN AS INTEGER)
    AND indicators.dist = CAST (geometry.CDIS AS INTEGER)
    AND indicators.secc = CAST (geometry.CSEC AS INTEGER)
GROUP BY geometry.NPRO
ORDER BY percentage DESC
LIMIT 10;
"""

conn = None
try:
    conn = psycopg2.connect(host="localhost",database="carto", user="postgres", password="postgres")
    cur = conn.cursor()
    cur.execute(command)
    res = cur.fetchall() 
    
    for row in res:
        print("Province = {}".format(row[0]) )
        print("% people with third studies = {:0.2f}%\n".format(row[1]) )
    
    cur.close()
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()

Province = Gipuzcoa
% people with third studies = 48.43%

Province = Madrid
% people with third studies = 23.29%

Province = Bizkaia
% people with third studies = 19.30%

Province = Valladolid
% people with third studies = 18.24%

Province = Gipuzkoa
% people with third studies = 18.17%

Province = Barcelona
% people with third studies = 18.03%

Province = Araba/Álava
% people with third studies = 17.83%

Province = Navarra
% people with third studies = 17.60%

Province = Salamanca
% people with third studies = 17.31%

Province = Zaragoza
% people with third studies = 16.90%

